In [1]:
import numpy as np
from PIL import Image
import os
import glob

In [2]:
filesimg = glob.glob('C:/Users/hakki.arkangil/Downloads/tuborg_yolov3_etiketli_veri' + '/**/*.jpg', recursive=True)
filestxt = glob.glob('C:/Users/hakki.arkangil/Downloads/tuborg_yolov3_etiketli_veri' + '/**/*.txt', recursive=True)

zipped= zip(filesimg ,filestxt)

zippath=list(zipped)

In [3]:
osn=os.path.basename(zippath[0][0])
osn

len (zippath)
zippath[378]

('C:/Users/hakki.arkangil/Downloads/tuborg_yolov3_etiketli_veri\\IMG_437311.JPG',
 'C:/Users/hakki.arkangil/Downloads/tuborg_yolov3_etiketli_veri\\IMG_437311.txt')

In [4]:
def croplabeler(idx,zippath):
    
    osim=os.path.basename(zippath[idx][0])
    ostxt=os.path.basename(zippath[idx][1])
    
    img = Image.open(osim)
    width,height= img.width, img.height

    f= open(ostxt,"r")
    annots= f.read().splitlines()

    count=0
    imgs=[]
    label=[]
    for annot in annots:
        fields = annot.split(' ')
        label.append(fields[0])
    
        annot = list(map(float,annot.split(" ")))[1:]
        x,y,w,h= annot[0] * width ,annot[1] * height, annot[2] * width ,annot[3] * height
         
        tl_x,tl_y,bt_rx,bt_ry = x-(w/2), y-(h/2) , x+(w/2) , y+(h/2)
        cropped = img.crop((tl_x,tl_y,bt_rx,bt_ry))
        cropped.save("C:/Users/hakki.arkangil/Desktop/crops/" + str(idx)+ str(count) +".jpg", 'JPEG')
        #print("my position is : " + zippath[idx][0])
        imgs.append(cropped)  #sikinti 26 lik dondurukten sonra ustuen yaziyo arrayin
        count+=1
        
        
    return imgs,label
## takes multiple object image as input and outputs each object alone

In [5]:
imgbasket=[]
labelbasket=[]
for i in range (len(zippath)):
    imgs,labels=croplabeler(i,zippath)
    imgbasket.append(imgs)
    labelbasket.append(labels)

SystemError: tile cannot extend outside image

In [6]:
flat_list_img = []
for sublist in imgbasket:
    for item in sublist:
        flat_list_img.append(item)

In [7]:
flat_list_label = []
for sublist in labelbasket:
    for item in sublist:
        flat_list_label.append(item)

In [8]:
import cv2

In [9]:
flat_list_img_np=[]
for i in flat_list_img:                              ##convert images to numpy array 
    temp=np.asarray(i)
    flat_list_img_np.append(temp)

In [10]:
flat_list_img_g=[]
for i in flat_list_img_np:
    temp=cv2.cvtColor(np.float32(i), cv2.COLOR_RGB2GRAY)    # make the numpy array one dimensional
    temp = temp[..., None]
    flat_list_img_g.append(temp) 

In [11]:
## give images the lost dimension back !!

In [12]:
uff=np.asarray(flat_list_img_g)

In [13]:
uff.shape

(8251,)

In [14]:
### PIL images must be array
X=[]
for i in flat_list_img_g:
    a=cv2.resize(i,(28,28))
    a = a[..., None]
    X.append(a)                                                   ### 28x28
    

In [15]:
ya=np.asarray(X)

In [16]:
ya.shape

(8251, 28, 28, 1)

In [17]:
ya=np.swapaxes(ya,1,3)

In [18]:
ya.shape

(8251, 1, 28, 28)

In [19]:
 # ya.reshape(8251,200,50,-1) reshape yemedi

In [136]:
Y=flat_list_label

In [144]:
Y = [int(numeric_string) for numeric_string in Y]


In [145]:
Y = np.asarray(Y)

In [23]:
#normalization...


def normalize2(feature): # normalization to 0-1 range
    min_feat = np.min(feature)
    max_feat = np.max(feature)
    feature = (feature - min_feat) / (max_feat-min_feat)
    return feature

In [24]:
X_t = normalize2(ya)  

In [25]:
X_t = np.asarray(X_t)

In [74]:
from modAL.models import ActiveLearner
from modAL.uncertainty import uncertainty_sampling

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [75]:
n_initial = 500

In [164]:
X_t.shape

(8251, 1, 28, 28)

In [165]:
X_t=X_t.astype(np.float32)

In [224]:
Y=Y.astype(np.longlong)

In [225]:
 X_train, X_test, y_train, y_test = train_test_split(X_t, Y)            #here

In [226]:
type(y_test[1])

numpy.int64

In [227]:
type(X_train[1][0][0][0])

numpy.float32

In [228]:
type(X_test[1][0][0][0])

numpy.float32

In [229]:
initial_idx = np.random.choice(range(len(X_train)), size=n_initial, replace=False)


In [268]:
X_train.shape

(6188, 1, 28, 28)

In [269]:
y_train= np.zeros(6188)

In [270]:
y_train[2300:4200]=1

In [271]:
y_train[172:994]=1

In [275]:
y_train=y_train.astype(np.longlong)

In [276]:
X_initial, y_initial = X_train[initial_idx], y_train[initial_idx]
X_pool, y_pool = np.delete(X_train, initial_idx, axis=0), np.delete(y_train, initial_idx, axis=0)




In [277]:
X_initial.shape

(500, 1, 28, 28)

In [278]:
y_initial.shape

(500,)

In [279]:
y_initial=y_initial.astype(np.longlong)

In [280]:
X_val, y_val= X_train[0:300], y_train[0:300]

In [281]:
type(y_val[2])

numpy.int64

In [282]:
###veriseti tamam training routuine hazirla

In [283]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

from modAL.models import ActiveLearner, BayesianOptimizer
from scipy import stats
from scipy.ndimage.filters import gaussian_filter1d
from scipy.spatial.distance import pdist, squareform
from skorch import NeuralNetClassifier
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split

In [284]:
def predictions_from_pool(
    model, X_pool: np.ndarray, T: int = 100, training: bool = True
):
    """Run random_subset prediction on model and return the output
    Attributes:
        X_pool: Pool set to select uncertainty,
        T: Number of MC dropout iterations aka training iterations,
        training: If False, run test without MC dropout. (default=True)
    """
    random_subset = np.random.choice(range(len(X_pool)), size=500, replace=False)
    with torch.no_grad():
        outputs = np.stack(
            [
                torch.softmax(
                    model.estimator.forward(X_pool[random_subset], training=training),
                    dim=-1,
                )
                .cpu()
                .numpy()
                for _ in range(T)
            ]
        )
    return outputs, random_subset

In [285]:
def shannon_entropy_function(
    model, X_pool: np.ndarray, T: int = 100, E_H: bool = False, training: bool = True
):
    """H[y|x,D_train] := - sum_{c} p(y=c|x,D_train)log p(y=c|x,D_train)
    Attributes:
        model: Model that is ready to measure uncertainty after training,
        X_pool: Pool set to select uncertainty,
        T: Number of MC dropout iterations aka training iterations,
        E_H: If True, compute H and EH for BALD (default: False),
        training: If False, run test without MC dropout. (default=True)
    """
    outputs, random_subset = predictions_from_pool(model, X_pool, T, training=training)
    pc = outputs.mean(axis=0)
    H = (-pc * np.log(pc + 1e-10)).sum(
        axis=-1
    )  # To avoid division with zero, add 1e-10
    if E_H:
        E = -np.mean(np.sum(outputs * np.log(outputs + 1e-10), axis=-1), axis=0)
        return H, E, random_subset
    return H, random_subset

In [286]:
def bald(
    model, X_pool: np.ndarray, n_query: int = 10, T: int = 100, training: bool = True
):
    """Choose pool points that are expected to maximise the information
    gained about the model parameters, i.e. maximise the mutal information
    between predictions and model posterior. Given
    I[y,w|x,D_train] = H[y|x,D_train] - E_{p(w|D_train)}[H[y|x,w]]
    with w the model parameters (H[y|x,w] is the entropy of y given w).
    Points that maximise this acquisition function are points on which the
    model is uncertain on average but there exist model parameters that produce
    disagreeing predictions with high certainty. This is equivalent to points
    with high variance in th einput to the softmax layer
    Attributes:
        model: Model that is ready to measure uncertainty after training,
        X_pool: Pool set to select uncertainty,
        n_query: Number of points that maximise bald a(x) from pool set,
        T: Number of MC dropout iterations aka training iterations,
        training: If False, run test without MC dropout. (default=True)
    """
    H, E_H, random_subset = shannon_entropy_function(
        model, X_pool, T, E_H=True, training=training
    )
    acquisition = H - E_H
    idx = (-acquisition).argsort()[:n_query]
    query_idx = random_subset[idx]
    return query_idx, X_pool[query_idx]

In [287]:
class ConvNN(nn.Module):
    def __init__(
        self,
        num_filters: int = 32,
        kernel_size: int = 4,
        dense_layer: int = 128,
        img_rows: int = 28,
        img_cols: int = 28,
        maxpool: int = 2,
    ):
        """
        Basic Architecture of CNN
        Attributes:
            num_filters: Number of filters, out channel for 1st and 2nd conv layers,
            kernel_size: Kernel size of convolution,
            dense_layer: Dense layer units,
            img_rows: Height of input image,
            img_cols: Width of input image,
            maxpool: Max pooling size
        """
        super(ConvNN, self).__init__()
        self.conv1 = nn.Conv2d(1, num_filters, kernel_size, 1)
        self.conv2 = nn.Conv2d(num_filters, num_filters, kernel_size, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(
            num_filters
            * ((img_rows - 2 * kernel_size + 2) // 2)
            * ((img_cols - 2 * kernel_size + 2) // 2),
            dense_layer,
        )
        self.fc2 = nn.Linear(dense_layer, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        out = self.fc2(x)
        return out

In [288]:
def active_learning_procedure(
    query_strategy,
    X_val: np.ndarray,
    y_val: np.ndarray,
    X_test: np.ndarray,
    y_test: np.ndarray,
    X_pool: np.ndarray,
    y_pool: np.ndarray,
    X_init: np.ndarray,
    y_init: np.ndarray,
    estimator,
    T: int = 100,
    n_query: int = 10,
    training: bool = True,
):
    """Active Learning Procedure
    Attributes:
        query_strategy: Choose between Uniform(baseline), max_entropy, bald,
        X_val, y_val: Validation dataset,
        X_test, y_test: Test dataset,
        X_pool, y_pool: Query pool set,
        X_init, y_init: Initial training set data points,
        estimator: Neural Network architecture, e.g. CNN,
        T: Number of MC dropout iterations (repeat acqusition process T times),
        n_query: Number of points to query from X_pool,
        training: If False, run test without MC Dropout (default: True)
    """
    learner = ActiveLearner(
        estimator=estimator,
        X_training=X_init,
        y_training=y_init,
        query_strategy=query_strategy,
    )
    perf_hist = [learner.score(X_test, y_test)]
    for index in range(T):
        query_idx, query_instance = learner.query(
            X_pool, n_query=n_query, T=T, training=training
        )
        learner.teach(X_pool[query_idx], y_pool[query_idx])
        X_pool = np.delete(X_pool, query_idx, axis=0)
        y_pool = np.delete(y_pool, query_idx, axis=0)
        model_accuracy_val = learner.score(X_val, y_val)
        if (index + 1) % 5 == 0:
            print(f"Val Accuracy after query {index+1}: {model_accuracy_val:0.4f}")
        perf_hist.append(model_accuracy_val)
    model_accuracy_test = learner.score(X_test, y_test)
    print(f"********** Test Accuracy per experiment: {model_accuracy_test} **********")
    return perf_hist, model_accuracy_test

In [289]:
ACQ_FUNCS = [bald]
BATCH_SIZE = 128
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
DROPOUT_ITER = 100
EPOCHS = 50
EXPERIMENTS = 3
N_QUERY = 10
LR = 1e-3
RESULT_DIR = "result_npy"
SEED = 369


if not os.path.exists(RESULT_DIR):
    os.mkdir(RESULT_DIR)


print(f"Using device: {DEVICE}")

Using device: cpu


In [290]:
def load_CNN_model():
    """Load new model each time for different acqusition function
    each experiments"""
    model = ConvNN().to(DEVICE)
    cnn_classifier = NeuralNetClassifier(
        module=model,
        lr=LR,
        batch_size=BATCH_SIZE,
        max_epochs=EPOCHS,
        criterion=nn.CrossEntropyLoss,
        optimizer=torch.optim.Adam,
        train_split=None,
        verbose=0,
        device=DEVICE,
    )
    return cnn_classifier


def save_as_npy(data: np.ndarray, name: str):
    """Save result as npy file
    
    Attributes:
        data: np array to be saved as npy file,
        folder: result folder name,
        name: npy filename
    """
    file_name = os.path.join(RESULT_DIR, name+".npy")
    np.save(file_name, data)
    print(f"Saved: {file_name}")


def plot_results(data: dict):
    """Plot results histogram using matplotlib"""
    sns.set()
    plt.figure(figsize=(7,5))
    for key in data.keys():
        data[key] = gaussian_filter1d(data[key], sigma=0.9) # for smoother graph
        plt.plot(data[key], label=key)
    plt.ylim([0.94, 1.00])
    plt.legend()
    plt.show()


def print_elapsed_time(start_time: float, exp: int, acq_func: str):
    """Print elapsed time for each experiment of acquiring
    Attributes:
        start_time: Starting time (in time.time()),
        exp: Experiment iteration
        acq_func: Name of acquisition function
    """
    elp = time.time() - start_time
    print(
        f"********** Experiment {exp} ({acq_func}): {int(elp//3600)}:\
        {int(elp%3600//60)}:{int(elp%60)} **********")

In [291]:
results = dict()


for i, acq_func in enumerate(ACQ_FUNCS):
    avg_hist = []
    test_scores = []
    acq_func_name = str(acq_func).split(" ")[1]
    print(f"\n---------- Start {acq_func_name} training! ----------")
    for e in range(EXPERIMENTS):
        start_time = time.time()
        estimator = load_CNN_model()
        print(
            f"********** Experiment Iterations: {e+1}/{EXPERIMENTS} **********"
        )
        training_hist, test_score = active_learning_procedure(
            query_strategy=acq_func,
            X_val=X_val,
            y_val=y_val,
            X_test=X_test,
            y_test=y_test,
            X_pool=X_pool,
            y_pool=y_pool,
            X_init=X_initial,
            y_init=y_initial,
            estimator=estimator,
            T=DROPOUT_ITER,
            n_query=N_QUERY
        )
        avg_hist.append(training_hist)
        test_scores.append(test_score)
        print_elapsed_time(start_time, e + 1, acq_func_name)
    avg_hist = np.average(np.array(avg_hist), axis=0)
    avg_test = sum(test_scores) / len(test_scores)
    print(f"Average Test score for {acq_func_name}: {avg_test}")
    results[acq_func_name] = avg_hist
    save_as_npy(data=avg_hist, name=acq_func_name)


print("--------------- Done Training! ---------------")


---------- Start bald training! ----------
********** Experiment Iterations: 1/3 **********
Val Accuracy after query 5: 0.5800
Val Accuracy after query 10: 0.5833
Val Accuracy after query 15: 0.5867


KeyboardInterrupt: 